In [6]:
#Loading packages
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import netCDF4

In [16]:
#Function to plot any number of variables
def plot_pollutant(variable,day):
    path='/proj/ie/proj/CMAS/CMAQ/cyclecloud-cmaq/notebook'
    filename = 'CCTM_ACONC_v54_cb6r5_ae7_aq_WR413_MYR_gcc_2018_12US1_3x160_20x24_beeond_201712{}.nc'.format(day)
    gridcro2d='GRIDCRO2D_20171222.nc'
    filepath = os.path.join(path, filename)
    gridname =os.path.join(path,gridcro2d)
    ds = netCDF4.Dataset(filepath, 'r')
    ds1 = netCDF4.Dataset(gridname, 'r')
    plt.rcParams['font.family'] = 'serif'
    File1 = ds
    #projection of our CMAQ netcdf file
    pproj = ccrs.LambertConformal(
        central_longitude=File1.XCENT,
        central_latitude=File1.YCENT,
        false_easting=0.0,
        false_northing=0.0,
        globe=None,
        cutoff=-30)
    x = np.arange(
        start=File1.XORIG,
        stop=File1.XORIG + (File1.NCOLS + 1) * File1.XCELL,
        step=File1.XCELL)

    y = np.arange(
        start=File1.YORIG,
        stop=File1.YORIG + (File1.NROWS + 1) * File1.YCELL,
        step=File1.YCELL)
    xx, yy = np.meshgrid(x, y)
        
    lat = ds1.variables['LAT'][0, 0, :, :]
    lon = ds1.variables['LON'][0, 0, :, :]
    MASK = ds1.variables['LWMASK'][0,0,:,:]
    MASK = np.squeeze(MASK)
    data = ds.variables[variable][:,0,:,:].mean(axis=0)  # Assuming time is the first dimension Mean of daily data
    # Mask the data where MASK is 0
    data_mean = np.ma.masked_where(MASK == 0, data) #plotting data only over Land
    #To find the minimum and maximum of latitude and longitude to get extent
    lat1 = lat.min()
    lat2 = lat.max()
    lon1 = lon.min()
    lon2 = lon.max()

    fig_width = 10
    fig_height = 8
    fig, ax = plt.subplots(subplot_kw={'projection': pproj}, figsize=(fig_width, fig_height))
    plt.rcParams['font.family'] = 'serif'
    ease_extent = [x[0], x[-1], y[0], y[-1]]
    cmap = plt.get_cmap('Spectral_r')
    cs = ax.pcolormesh(xx, yy, data_mean, vmin=np.nanmin(data_mean), vmax=np.nanmax(data_mean), cmap=cmap)  # Compute mean along time axis
    plt.title('Mean {}_201712{}'.format(variable,day),fontsize=14,loc='center')
    ax.set_extent(ease_extent, crs=pproj)
    ax.gridlines(color='gray', linestyle='--')
    ax.gridlines(draw_labels=True, x_inline=False, y_inline=False, linewidth=0.2)
    #adding state shapefile with the help of cartopy
    ax.add_feature(cfeature.STATES.with_scale('10m'), linewidth=0.3)
    #adding ocean layer 
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_feature(cartopy.feature.LAKES)
    #ax.add_feature(cartopy.feature.LAND,rasterized=True)
    fn = shpreader.natural_earth(resolution='10m', category='cultural', name='admin_1_states_provinces')
    reader = shpreader.Reader(fn)
    states = [x for x in reader.records() if x.attributes["adm0_a3"] == "USA"]

    data_proj = ccrs.PlateCarree()
    ax.set_extent([lon1, lon2, lat1, lat2], crs=data_proj)
    #For loop below with plot the name of States in code North Carolina : NC
    exclude_states = ['AK', 'HI']
    for state in states:
        if state.attributes["postal"] not in exclude_states:
            lon = state.geometry.centroid.x
            lat = state.geometry.centroid.y
            name = state.attributes["postal"]

            ax.text(
                lon, lat, name, size=4, transform=data_proj, ha="center", va="center",
                path_effects=[PathEffects.withStroke(linewidth=3, foreground=(1, 1, 1, 0))]
            )

    cax = fig.add_axes([0.95, 0.2, 0.02, 0.6])  # [left, bottom, width, height]
    cbar = plt.colorbar(cs, cax=cax, orientation='vertical')
    cbar.set_label('ppmv')
    name='Mean_{}_201712{}.png'.format(variable,day)
    plots = os.path.join(path, 'plots/')
    os.makedirs(plots, exist_ok=True)
    plt.savefig(os.path.join(plots,name), bbox_inches="tight", dpi=300)
    plt.close()



In [17]:
#Main program 
variables = ["O3", "NO2"] # creating the list of variables to plot
days = ["22"] # if you have hourly data in daily files. 
#path='/proj/ie/proj/CMAS/CMAQ/cyclecloud-cmaq/notebook'
# Call the function for each variable and day
for var in variables:
    for day in days:
        plot_pollutant(var, day)